# probability of the real structure,  under different SHAPE arrays

In [8]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
backend module://ipykernel.pylab.backend_inline version unknown


# Load data

In [9]:
import rna_tools.rna_io as rio

def getdata():
    return rio.get_all_data("../data/RNA16.react","../data/RNA16.dbn")  # {key: rea, seq, stru}

data = getdata()

for k,v in data.items():print k,"\t",  len(v[1])

TRP5 	202
5srRNA 	170
GLYCFN 	205
RNAsep 	154
adenin 	71
Z-CIDGMP-1 	135
R009 	1542
ZHCV 	95
MDLOOP 	80
srRNA 	120
p564 	155
tRNAphe 	76
ZGLY 	198
23sRNA 	2904
ADDRSW 	121
tRNA-asp 	75


# TRAIN , get proba of structures

In [14]:
from rna_tools.sukosd import sukosd
from rna_tools.structureprobability import probabilities_of_structures
import simushape as sim

data = getdata()
model  = sim.make_model(data,['23sRNA','R009'], model= sim.make_forestregressor())
data.pop("23sRNA")
data.pop("R009")

def get_str(seq):
    for _, data_seq, db in data.values():
        if seq == data_seq:
            return [db]
    print "SHIT"

#predictions = { k: (v[1],sim.predict(model,v[1],seq_to_db_function=get_str)) for k,v in data.items()  }
predictions = { k: (v[1],sim.predict(model,v[1])) for k,v in data.items()  }

keys = predictions.keys()


prb_suko =   [probabilities_of_structures(data[k][1], [data[k][2]], sukosd(data[k][2]))[0][1] for  k in keys] # input: seq and str, returns probability (sukosd(data[k][2]) returns shape)
prb_real =   [probabilities_of_structures(data[k][1],[data[k][2]], data[k][0])[0][1] for  k in keys] #rnaplfold gets sequence and shape, returns probab 
prb_shaker = [probabilities_of_structures(data[k][1],[data[k][2]], predictions[k])[0][1] for k in keys] # a predcition is seq,rea # probability




# Visualize 

In [15]:
import pandas
import numpy as np
index= ['suko',"shaker", "real"]
data2 = [prb_suko,prb_shaker, prb_real ] # data is a list 

map(lambda x:x.append(np.mean(x)),data2)
df = pandas.DataFrame(data2, columns=keys+['mean'], index=index).T
df

,suko,shaker,real
TRP5,8.532152e-10,2.212920e-12,1.096787e-15
5srRNA,7.934274e-04,3.806892e-07,1.829143e-09
GLYCFN,2.423783e-05,7.900716e-07,5.845614e-25
RNAsep,5.394035e-13,1.064771e-13,3.144387e-16
adenin,5.225610e-01,2.477391e-01,5.058758e-01
Z-CIDGMP-1,7.062428e-06,8.826028e-09,2.976081e-09
ZHCV,4.346807e-05,2.271472e-05,8.556295e-08
MDLOOP,3.716712e-01,1.014923e-01,2.820767e-01
srRNA,7.404234e-07,5.432274e-08,1.010630e-04
p564,1.822734e+00,3.756221e-05,3.777509e-01


# BUT

this is  not mega nice. 
so ... 
1. try to show energies instead of proba, because the probas seem messed up.. 
2. compare with no-data given

In [5]:
from rna_tools.structureprobability import get_stru_energy

prb_suko =   [get_stru_energy(data[k][2],data[k][1], sukosd(data[k][2])) for  k in keys] # input: seq and str, returns probability (sukosd(data[k][2]) returns shape)
prb_real =   [get_stru_energy(data[k][2],data[k][1], data[k][0]) for  k in keys] #rnaplfold gets sequence and shape, returns probab 
prb_shaker = [get_stru_energy(data[k][2],data[k][1], predictions[k]) for k in keys] # a predcition is seq,rea # probability
prb_nodata = [get_stru_energy(data[k][2],data[k][1]) for k in keys] # a predcition is seq,rea # probability



In [6]:
import pandas
import numpy as np
index= ['suko',"shaker", "real",'nothing']
#data = [corr_suko,corr_shaker]
data2 = [prb_suko,prb_shaker, prb_real,prb_nodata ] # data is a list 

map(lambda x:x.append(np.mean(x)),data2)
df = pandas.DataFrame(data2, columns=keys+['mean'], index=index).T

In [7]:
print df

                 suko  shaker   real    nothing
TRP5       -80.770000  -43.10 -87.00 -43.100000
5srRNA     -76.610000  -45.40 -76.31 -45.400000
GLYCFN     -69.440000  -26.00 -40.68 -26.000000
RNAsep     -77.190000  -30.88 -73.75 -30.200000
adenin     -42.960000  -18.58 -40.06 -18.500000
Z-CIDGMP-1 -47.210000  -21.00 -47.60 -21.000000
ZHCV       -36.720000  -19.70 -38.87 -19.700000
MDLOOP     -25.350000  -14.40 -16.13 -14.400000
srRNA      -73.900000  -43.80 -53.72 -42.700000
p564       -88.830000  -23.70 -91.06 -22.800000
tRNAphe    -44.160000  -23.44 -61.73 -22.400000
tRNA-asp   -42.710000  -27.24 -50.08 -26.400000
ADDRSW     -47.310000  -19.60 -45.42 -19.600000
ZGLY       -51.880000  -24.10 -72.23 -24.100000
mean       -57.502857  -27.21 -56.76 -26.878571
